In [30]:
import requests
import pandas as pd
from sqlalchemy import create_engine
import sqlite3
from requests.auth import HTTPBasicAuth

In [44]:
# Step 1: Extract Data from API in Chunks
def fetch_data_in_chunks(api_url, params, start, end, chunk_size=1000):
    for i in range(start, end, chunk_size):
        params['start'] = i
        params['end'] = i + chunk_size

        # Make the API request with HTTP Basic Authentication
        response = requests.get(api_url, params=params, auth=HTTPBasicAuth(username, password))
        
        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()  # Parse the JSON response
            if not data:
                break  # Exit loop if no more data is returned
            all_data.extend(data)  # Add the retrieved data to the list
            offset += limit        # Increment the offset for the next request
        else:
            print(f'Error: {response.status_code}, {response.text}')
            break


In [47]:
# API details
api_url = "https://data.colorado.gov/resource/4ykn-tg5h.json"
username = "glissmann.mark@gmail.com"
password = "K2PRH@mUDwgbfsV"
date_filter = "entityformdate >= '2023-01-01T00:00:00'"
# Set the parameters for pagination
limit = 1000  # Number of records per request
offset = 0    # Starting offset
params = {
        '$where': date_filter
    }

# Define the range and chunk size
start = 0
end = 10000 
chunk_size = 1000

# Database engine
engine = create_engine('sqlite:///business_entities.db')




In [46]:
# Step 2: Load Data into Database in Chunks
for df_chunk in fetch_data_in_chunks(api_url, params, start, end, chunk_size):
    df_chunk.to_sql('colorado', con=engine, if_exists='append', index=False)
    print(f"Data chunk {params['start']} to {params['end']} loaded into the database successfully!")


Error: 400, {
  "error" : true,
  "message" : "Unrecognized arguments [end, start]"
}



TypeError: 'NoneType' object is not iterable

In [26]:
# Step 3: Query Data Using SQL
def query_data_from_db(db_path, query):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Execute the query
    cursor.execute(query)
    
    # Fetch the results and load into a DataFrame
    results = cursor.fetchall()
    columns = [description[0] for description in cursor.description]
    result_df = pd.DataFrame(results, columns=columns)
    
    # Close the connection
    conn.close()
    
    return result_df


In [27]:
# Define the database path and SQL query
db_path = 'weather_data.db'
query = "SELECT * FROM weather"

# Execute the query and print the results
result_df = query_data_from_db(db_path, query)
print(result_df)


OperationalError: no such table: weather